This notebook was used to train and save the classifier.


In [ ]:
import sys
import random
import numpy as np
import pandas as pd
import skimage.morphology as morph
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from skimage.feature import local_binary_pattern, hog
from sklearn.ensemble import RandomForestClassifier
from google.colab import drive
import matplotlib.pyplot as plt
import pickle

We are using a dataframe that has in each row a grayscale image.

In [ ]:
df = pd.read_csv('dataset.csv', sep=",", header=0)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,22461,22462,22463,22464,22465,22466,22467,22468,22469,22470,22471,22472,22473,22474,22475,22476,22477,22478,22479,22480,22481,22482,22483,22484,22485,22486,22487,22488,22489,22490,22491,22492,22493,22494,22495,22496,22497,22498,22499,22500
0,8,9,9,7,7,10,10,12,12,13,13,15,13,10,12,10,10,11,10,10,9,9,9,13,15,17,17,18,20,19,19,17,18,16,15,20,18,21,20,21,...,51,56,60,61,78,51,60,66,53,62,67,52,64,64,58,66,72,58,62,62,61,65,63,54,61,53,54,60,56,50,51,49,49,45,49,46,28,40,37,1
1,8,8,6,6,10,10,7,7,7,9,9,6,9,7,6,9,10,7,7,10,10,10,9,7,7,7,7,6,9,8,9,8,7,9,7,10,10,10,11,11,...,40,39,45,43,36,41,50,48,49,41,46,51,48,41,46,36,45,43,47,47,42,44,39,37,41,41,42,37,33,32,32,32,33,33,23,28,27,29,24,1
2,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,1
3,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,1
4,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,111,6,7,7,6,9,8,8,8,...,31,29,29,29,29,29,29,124,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,3
888,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,156,85,87,90,91,87,91,91,91,...,114,111,105,110,107,99,92,156,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,3
889,81,80,80,85,86,86,85,89,86,86,88,87,91,91,92,87,85,90,92,87,90,91,100,87,96,96,97,95,94,99,97,101,96,90,101,102,89,97,104,101,...,58,52,51,59,39,0,8,33,64,78,84,97,107,104,120,113,124,119,125,126,129,127,131,124,129,125,124,131,127,130,135,135,132,135,141,139,138,136,139,3
890,139,142,141,140,137,134,140,138,136,136,140,140,140,131,135,136,134,130,128,134,128,124,126,119,123,120,118,116,112,108,107,95,88,86,81,78,74,67,55,40,...,94,92,92,92,91,93,91,91,89,90,94,92,89,90,87,90,85,79,93,92,91,93,87,88,89,86,89,81,79,85,86,86,82,82,80,81,83,83,82,3


In [ ]:
def median_filter(img, filterSize=3):
  filterRadius = filterSize//2
  img_pad = np.pad(img, ((filterRadius, ),(filterRadius, )), 'edge')


  filtered_img = np.zeros(img_pad.shape)
  for x in np.arange(filterRadius, img_pad.shape[0]-filterRadius+1):
    for y in np.arange(filterRadius, img_pad.shape[1]-filterRadius+1):
      med_region = np.median(img_pad[x-filterRadius:x+filterRadius+1, y-filterRadius:y+filterRadius+1])
      filtered_img[x,y] = med_region

  return filtered_img[filterRadius:img_pad.shape[0]-filterRadius, filterRadius:img_pad.shape[1]-filterRadius]

In [ ]:
def preprocessing(img):
  aux = np.reshape(img, (150, 150))
  aux = median_filter(aux)
  aux = aux - morph.erosion(aux, morph.disk(2))

  return aux


In [ ]:
def get_mean_accuracy(model, X, y):
  
  random.seed(10)
  accuracy = 0.0
  cv = StratifiedKFold(n_splits=10, shuffle=False)


  for train_index, test_index in cv.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train, y_train.reshape(-1))
    y_pred = model.predict(X_test)

    accuracy += accuracy_score(y_test.reshape(-1), y_pred)
  return float(accuracy/10)

In [ ]:
#Slipting data from classes
data = df.iloc[:, :-1].to_numpy()
classes = df.iloc[:,-1].to_numpy()
data_img = []

#processing the data to extract the features
for i in range(data.shape[0]):
  data_img.append(preprocessing(data[i]))

In [ ]:
#extracting the features
lbp = []

for i in range(len(data_img)):
  aux = local_binary_pattern(data_img[i], 8, 1)
  hist, bins = np.histogram(aux, int(aux.max() + 1), density=True)
  lbp.append(hist)

In [ ]:
#extracting the features
hog_v = []
for i in range(len(data_img)): 
  hog_v.append(hog(data_img[i], feature_vector=True))

In [ ]:
data_features = np.empty(len(data_img), dtype=np.ndarray)
for i in range(len(data_img)):
  data_features[i] = np.append(lbp[i],hog_v[i])

data_features = np.vstack(data_features)

In [ ]:
#trainig the model and acquiring the mean accuracy
mlp = MLPClassifier(random_state=10)

acc = get_mean_accuracy(mlp, data_features, classes)
print(acc)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.7206491885143571


In [ ]:
#Testing another model for comparation
rf = RandomForestClassifier(random_state=0)
acc2 = get_mean_accuracy(rf, data_features, classes)
print(acc2)

0.6217852684144818


In [ ]:
#Saving the MLP model
filename = 'finalized_model.sav'
pickle.dump(mlp, open(filename, 'wb'))